In [3]:
from abc import ABC, abstractmethod

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class BenchmarkFunction(ABC):
    """Interfaz para problemas de optimización."""

    def __init__(self, dimension: int):
        self.dimension = dimension

    @abstractmethod
    def evaluate(self, position: np.ndarray) -> float:
        """Evalúa una solución (posición)."""
        pass

    @abstractmethod
    def get_bounds(self) -> tuple[float, float]:
        """Devuelve los límites inferior y superior del espacio de búsqueda."""
        pass


class Krill:
    """Representa un krill en el enjambre."""

    def __init__(self, position: np.ndarray):
        self.position = position
        self.fitness = np.inf

    def update_fitness(self, problem: BenchmarkFunction):
        self.fitness = problem.evaluate(self.position)


class Population:
    """Gestiona el enjambre de krill."""

    def __init__(self, size: int, problem: BenchmarkFunction, rng: np.random.Generator):
        self.problem = problem
        lower, upper = problem.get_bounds()
        self.krill = [
            Krill(rng.uniform(lower, upper, size=problem.dimension))
            for _ in range(size)
        ]
        for k in self.krill:
            k.update_fitness(problem)

    def get_best(self) -> Krill:
        return min(self.krill, key=lambda k: k.fitness)


class KHA:
    """Implementación del algoritmo Krill Herd."""

    def __init__(
        self, population_size: int, iterations: int, problem: BenchmarkFunction, seed: int
    ):
        self.rng = np.random.default_rng(seed)
        self.population = Population(population_size, problem, self.rng)
        self.iterations = iterations
        self.problem = problem

    def optimize(self):
        best = self.population.get_best()
        lower, upper = self.problem.get_bounds()

        for _ in range(self.iterations):
            new_positions: list[np.ndarray] = []

            for krill in self.population.krill:
                # --- Movimiento inducido por otros krill ---
                direction_social = best.position - krill.position

                # --- Movimiento de forrajeo ---
                direction_food = self.rng.uniform(-1, 1, size=self.problem.dimension)

                # --- Movimiento aleatorio ---
                direction_random = self.rng.normal(0, 0.1, size=self.problem.dimension)

                # Nueva posición (normalizada dentro de los límites)
                new_position = krill.position + 0.5 * (
                    direction_social + direction_food + direction_random
                )
                new_position = np.clip(new_position, lower, upper)

                new_positions.append(new_position)

            # Actualizar posiciones
            for krill, pos in zip(self.population.krill, new_positions):
                krill.position = pos
                krill.update_fitness(self.problem)

            # Actualizar mejor global
            current_best = self.population.get_best()
            if current_best.fitness < best.fitness:
                best = current_best

        return best


In [ ]:
# type: ignore

def plot_optimization(kha: KHA, problem: BenchmarkFunction):
    """Ilustra el proceso de optimización para funciones de 2 dimensiones."""
    if problem.dimension != 2:
        raise ValueError("La visualización solo está soportada para 2D")

    # Crear grilla para el paisaje de la función
    lower, upper = problem.get_bounds()
    x = np.linspace(lower, upper, 200)
    y = np.linspace(lower, upper, 200)
    X, Y = np.meshgrid(x, y)
    Z = np.array(
        [problem.evaluate(np.array([i, j])) for i, j in zip(X.ravel(), Y.ravel())]
    )
    Z = Z.reshape(X.shape)

    # Inicializar población
    population = kha.population

    plt.figure(figsize=(8, 6))
    cp = plt.contourf(X, Y, Z, levels=50, cmap="viridis")
    plt.colorbar(cp, label="Fitness")

    # Dibujar krill
    krill_x = [k.position[0] for k in population.krill]
    krill_y = [k.position[1] for k in population.krill]
    plt.scatter(krill_x, krill_y, c="red", label="Krill", edgecolors="black")

    # Dibujar mejor solución
    best = population.get_best()
    plt.scatter(
        best.position[0],
        best.position[1],
        c="yellow",
        marker="*",
        s=200,
        label="Mejor krill",
    )

    plt.title("Krill Herd Algorithm - Iteración inicial")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.legend()
    plt.show()


In [11]:
class Rastrigin(BenchmarkFunction):
    def __init__(self, dimension: int = 10):
        self.dimension = dimension

    def evaluate(self, position: np.ndarray) -> float:
        return 10 * self.dimension + np.sum(
            position**2 - 10 * np.cos(2 * np.pi * position)
        )

    def get_bounds(self) -> tuple[float, float]:
        return -5.12, 5.12


if __name__ == "__main__":
    # Definición del problema
    problem = Rastrigin(dimension=10)

    # Instanciación del algoritmo
    kha = KHA(population_size=30, iterations=200, problem=problem, seed=1234)

    # Optimización
    best_solution = kha.optimize()

    print("📌 Mejor posición encontrada:", best_solution.position)
    print("📉 Valor de la función:", best_solution.fitness)


📌 Mejor posición encontrada: [ 1.74063431  0.63168318  0.00334587 -0.08033292 -1.02857309  2.03157914
  3.07693438 -2.28017255 -0.61388079 -0.01347519]
📉 Valor de la función: 83.24008269080804
